<a href="https://colab.research.google.com/github/iatra/COSC2673-Assignment2-Group9/blob/main/s4005248_s3980067.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning COSC2673-Assignment2-Group9
Author : Joelle Lee Ming Yin (s3980067) & Isabella Tranquilino (s4005248)

# Load Dataset

In [28]:
from google.colab import drive
drive.mount('/content/drive')

# Both users update the path to point to the shared folder
zip_path = '/content/drive/MyDrive/ML A2 Dataset/Image_classification_data.zip'

import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/data')

import os

# List first few files/folders in the extraction folder
extracted_path = '/content/data'
for root, dirs, files in os.walk(extracted_path):
    print(f"Directory: {root}")
    for file in files[:5]:  # Show up to 5 files per directory
        print(f"  - {file}")
    break  # Remove this if you want to see all subfolders


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory: /content/data
  - data_labels_mainData.csv
  - data_labels_extraData.csv
